## Parameters that may be changed

In [ ]:
exp_name = 'Asn_24' # Asn_XX / Asn_XXX, Fc_DAO, Fc_EPO, or NN_modelNSD
activ_fun_list = ['relu', 'selu', 'tanh', 'tanhshrink'] # Entries should be in {'relu', 'tanh', 'sigmoid', 'tanhshrink', 'selu'}
single_glycan = 'GnGnGnF' # Predict only a single glycan at a time. Must set cutoff = 0 to use this. Set to None when using cutoff
nested_validation = False

## Other parameters (that likely shouldn't be changed) + imports

In [ ]:
cutoff = 0 # Ignore all glycans with a train mean < cutoff. Set to 0 when using single_glycan
weight_decay = 0 # Should leave equal to 0. weight_decay seldom makes a difference with these data
normalize_X = True # Whether to normalize the X data based on the training set mean and stdev
# normalize_y does not really work
normalize_y = False # Whether to normalize the y data based on the training set mean and stdev

# Quick input check - do not modify
if cutoff and single_glycan:
    raise ValueError('Cutoff must be 0 to use single_glycan. To use cutoff, set single_glycan to None')

In [ ]:
# General & data manipulation imports
import pandas as pd
import numpy as np
from os import mkdir
from os.path import isdir, isfile, dirname
from os.path import join as osjoin # join is too generic of a name to be imported directly
# Torch & model creation imports
import torch
from torch.utils.data import Dataset, DataLoader
from collections import OrderedDict
# Training & validation imports
from itertools import product
# Results & visualization imports
import matplotlib.pyplot as plt
# Convenience imports
import pdb

## Data Setup

In [ ]:
# Loading the data
if exp_name == 'NN_modelNSD':
    train_data_X = torch.Tensor(pd.read_csv(osjoin('datasets', 'NN_modelNSD_training-X.csv'), index_col = 0).values)
    test_data_X = torch.Tensor(pd.read_csv(osjoin('datasets', 'NN_modelNSD_test-X.csv'), index_col = 0).values)
else:
    train_data_X = torch.Tensor(pd.read_csv(osjoin('datasets', 'Training-X.csv'), index_col = 0).values)
    test_data_X = torch.Tensor(pd.read_csv(osjoin('datasets', 'Test-X.csv'), index_col = 0).values)
train_data_y = torch.Tensor(pd.read_csv(osjoin('datasets', f'{exp_name}_training-y.csv'), index_col = 0).values)
test_data_y = torch.Tensor(pd.read_csv(osjoin('datasets', f'{exp_name}_test-y.csv'), index_col = 0).values)

# Removing glycans based on cutoff
if cutoff:
    train_mean = train_data_y.mean(axis=0)
    above_cutoff = train_mean >= cutoff
    train_data_y = train_data_y[:, above_cutoff]
    test_data_y = test_data_y[:, above_cutoff]
elif single_glycan:
    columns = pd.read_csv(osjoin('datasets', f'{exp_name}_training-y.csv'), index_col = 0).columns
    glycan_idx = np.argwhere(columns == single_glycan)[0, 0]
    train_data_y = train_data_y[:, glycan_idx]
    test_data_y = test_data_y[:, glycan_idx]

# Nested validation: concatenating the training and test sets
if nested_validation:
    train_data_X = torch.cat((train_data_X, test_data_X))
    train_data_y = torch.cat((train_data_y, test_data_y))

# For convenience when declaring MLPs
myshape_X = train_data_X.shape[1]
if single_glycan:
    myshape_y = 1
else:
    myshape_y = train_data_y.shape[1]
# Pre-declaring paths for convenience (to save / load results)
if single_glycan:
    val_loss_file = f'ANN_{exp_name}_val-loss_{weight_decay:.1e}weight-decay_{single_glycan}.csv'
    best_model_file = f'ANN_{exp_name}_{weight_decay:.1e}weight-decay_{single_glycan}_dict.pt'
else:
    val_loss_file = f'ANN_{exp_name}_val-loss_{weight_decay:.1e}weight-decay_{cutoff}cutoff.csv'
    best_model_file = f'ANN_{exp_name}_{weight_decay:.1e}weight-decay_{cutoff}cutoff_dict.pt'
# Setting each activ_fun to lowercase for consistency
activ_fun_list = [activ_fun.casefold() for activ_fun in activ_fun_list]

In [ ]:
def generate_working_dir(activ_fun, normalize_X, normalize_y = False, nested_validation = False):
    # A helper function to compartimentalize results in labeled folders
    if normalize_y:
        working_dir = osjoin(f'ANN_normalized-y_results{"_nested" * nested_validation}',
                             f'ANN_results{"_nonormalization" * (not(normalize_X))}_{activ_fun}')
    else:
        working_dir = osjoin(f'ANN_default-y_results{"_nested" * nested_validation}',
                             f'ANN_results{"_nonormalization" * (not(normalize_X))}_{activ_fun}')
    # mkdir is not recursive
    first_dir = dirname(working_dir)
    if not isdir(first_dir):
        mkdir(first_dir)
    if not isdir(working_dir):
        mkdir(working_dir)
    return working_dir

In [ ]:
class MyDataset(Dataset):
    def __init__(self, Xdata, ydata):
        self.Xdata = Xdata
        self.ydata = ydata
    
    def __len__(self):
        return len(self.Xdata)
    
    def __getitem__(self, idx):
        return self.Xdata[idx], self.ydata[idx]

## Model & Run Setup

In [ ]:
# MLP model
class SequenceMLP(torch.nn.Module):
    def __init__(self, layers, activ_fun = 'relu'):
        super(SequenceMLP, self).__init__()
        # Setup to convert string (first cell) to activation function
        if activ_fun == 'relu':
            torch_activ_fun = torch.nn.ReLU()
        elif activ_fun == 'tanh':
            torch_activ_fun = torch.nn.Tanh()
        elif activ_fun == 'sigmoid':
            torch_activ_fun = torch.nn.Sigmoid()
        elif activ_fun == 'tanhshrink':
            torch_activ_fun = torch.nn.Tanhshrink()
        elif activ_fun == 'selu':
            torch_activ_fun = torch.nn.SELU()
        else:
            raise ValueError(f'Invalid activ_fun. You passed {activ_fun}')
        # Transforming layers list into OrderedDict with layers + activation
        mylist = list()
        for idx, elem in enumerate(layers):
            mylist.append((f'Linear{idx}', torch.nn.Linear(layers[idx][0], layers[idx][1]) ))
            if idx < len(layers)-1:
                mylist.append((f'{activ_fun}{idx}', torch_activ_fun))
        # OrderedDict into NN
        self.model = torch.nn.Sequential(OrderedDict(mylist))
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
# A helper function that is called every epoch of training or validation
def loop_model(model, optimizer, loader, epoch, evaluation = False):
    if evaluation:
        model.eval()
    else:
        model.train()
    batch_losses = []

    for data in loader:
        X, y = data
        X = X.cuda()
        y = y.cuda()
        pred = model(X).squeeze() # TODO: ensure squeeze also works when single_glycan == None
        loss = torch.nn.functional.mse_loss(pred, y)
        # Backpropagation
        if not evaluation:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        batch_losses.append(loss.item()) # Saving losses
    return np.array(batch_losses).mean()

In [ ]:
# Setting up the hyperparameters
if cutoff == 0 and not single_glycan: # Predict every glycan at the same time
    layers = [
        # 1 hidden layer
        [(myshape_X, myshape_X*9), (myshape_X*9, myshape_y)],
        [(myshape_X, myshape_X*7), (myshape_X*7, myshape_y)],
        [(myshape_X, myshape_X*5), (myshape_X*5, myshape_y)],
        [(myshape_X, myshape_X*3), (myshape_X*3, myshape_y)],
        [(myshape_X, myshape_X*2), (myshape_X*2, myshape_y)],
        # 2 hidden layers
        [(myshape_X, myshape_X*9), (myshape_X*9, myshape_X*9), (myshape_X*9, myshape_y)],
        [(myshape_X, myshape_X*9), (myshape_X*9, myshape_X*7), (myshape_X*7, myshape_y)],
        [(myshape_X, myshape_X*9), (myshape_X*9, myshape_X*5), (myshape_X*5, myshape_y)],
        [(myshape_X, myshape_X*9), (myshape_X*9, myshape_X*3), (myshape_X*3, myshape_y)],
        [(myshape_X, myshape_X*7), (myshape_X*7, myshape_X*7), (myshape_X*7, myshape_y)],
        [(myshape_X, myshape_X*7), (myshape_X*7, myshape_X*5), (myshape_X*5, myshape_y)],
        [(myshape_X, myshape_X*7), (myshape_X*7, myshape_X*3), (myshape_X*3, myshape_y)],
        [(myshape_X, myshape_X*5), (myshape_X*5, myshape_X*5), (myshape_X*5, myshape_y)],
        [(myshape_X, myshape_X*5), (myshape_X*5, myshape_X*3), (myshape_X*3, myshape_y)],
        [(myshape_X, myshape_X*3), (myshape_X*3, myshape_X*3), (myshape_X*3, myshape_y)],
        [(myshape_X, myshape_X*3), (myshape_X*3, myshape_X*2), (myshape_X*2, myshape_y)],
        [(myshape_X, myshape_X*2), (myshape_X*2, myshape_X*2), (myshape_X*2, myshape_y)],
        [(myshape_X, myshape_X*2), (myshape_X*2, myshape_X), (myshape_X, myshape_y)] ]
elif cutoff: # Predict a few glycans at the same time
    layers = [
        # 1 hidden layer
        [(myshape_X, myshape_X*4), (myshape_X*4, myshape_y)],
        [(myshape_X, myshape_X*3), (myshape_X*3, myshape_y)],
        [(myshape_X, myshape_X*2), (myshape_X*2, myshape_y)],
        [(myshape_X, myshape_X), (myshape_X, myshape_y)],
        # 2 hidden layers
        [(myshape_X, myshape_X*4), (myshape_X*4, myshape_X*4), (myshape_X*4, myshape_y)],
        [(myshape_X, myshape_X*4), (myshape_X*4, myshape_X*3), (myshape_X*3, myshape_y)],
        [(myshape_X, myshape_X*3), (myshape_X*3, myshape_X*3), (myshape_X*3, myshape_y)],
        [(myshape_X, myshape_X*3), (myshape_X*3, myshape_X*2), (myshape_X*2, myshape_y)],
        [(myshape_X, myshape_X*2), (myshape_X*2, myshape_X*2), (myshape_X*2, myshape_y)],
        [(myshape_X, myshape_X*2), (myshape_X*2, myshape_X), (myshape_X, myshape_y)],
        [(myshape_X, myshape_X), (myshape_X, myshape_X), (myshape_X, myshape_y)] ]
else: # Predict a single glycan at a time
    layers = [
        # 1 hidden layer
        [(myshape_X, myshape_X*2), (myshape_X*2, myshape_y)],
        [(myshape_X, myshape_X), (myshape_X, myshape_y)],
        [(myshape_X, myshape_X//2), (myshape_X//2, myshape_y)],
        # 2 hidden layers
        [(myshape_X, myshape_X*2), (myshape_X*2, myshape_X*2), (myshape_X*2, myshape_y)],
        [(myshape_X, myshape_X*2), (myshape_X*2, myshape_X), (myshape_X, myshape_y)],
        [(myshape_X, myshape_X), (myshape_X, myshape_X), (myshape_X, myshape_y)],
        [(myshape_X, myshape_X), (myshape_X, myshape_X//2), (myshape_X//2, myshape_y)] ]
lr_vals = [5e-2, 1e-2, 5e-3, 1e-3, 5e-4, 1e-4]
hyperparam_list = list(product(layers, lr_vals))
batch_size = 12

## Training and validating the model

In [ ]:
def CV_model(activ_fun, working_dir, val_loss_file, train_idx, val_idx):
    """
    This function runs a cross-validation procedure for each combination of layers + learning rates
    Results are saved in a .csv file inside {working_dir}
    """
    # Recording the validation losses
    try:
        final_val_loss = pd.read_csv(osjoin(f'{working_dir}', f'{val_loss_file}'), index_col = 0)
    except FileNotFoundError:
        final_val_loss = pd.DataFrame(np.nan, index = lr_vals, columns = [str(elem) for elem in layers])

    # Train and validate
    print(f'Beginning CV on activation function {activ_fun}')
    for cur_idx, cur_hp in enumerate(hyperparam_list):
        # We added a new layer configuration to the hyperparameters
        if not str(cur_hp[0]) in list(final_val_loss.columns):
            final_val_loss.insert(layers.index(cur_hp[0]), str(cur_hp[0]), np.nan) # layers.index to ensure consistent order
        # We added a new learning rate to the hyperparameters
        elif not cur_hp[1] in final_val_loss.index.to_list():
            final_val_loss.loc[cur_hp[1], :] = np.nan

        # Run CV only if we do not have validation losses for this set of parameters
        if np.isnan( final_val_loss.at[cur_hp[1], str(cur_hp[0])] ):
            if not (cur_idx+1) % 5: # Print only every 5 hyperparameters, since they go by pretty quickly
                print(f'Beginning hyperparameters {cur_idx+1:2}/{len(hyperparam_list)} for {activ_fun}', end = '\r')
            temp_val_loss = 0
            for fold_idx in range(4):
                #print(f'Current fold: {fold_idx+1}/4; layers = {cur_hp[0]}, lr = {cur_hp[1]}', end = '\r') # Folds are too short to bother printing
                # Separating the training data for this fold
                train_X_fold = train_data_X[train_idx[fold_idx], :]
                if single_glycan:
                    train_y_fold = train_data_y[train_idx[fold_idx]]
                else:
                    train_y_fold = train_data_y[train_idx[fold_idx], :]
                # Normalizing the training set
                if normalize_X:
                    mu, std = train_X_fold.mean(), train_X_fold.std()
                    train_X_fold = (train_X_fold - mu) / std
                if normalize_y:
                    mu_y, std_y = train_y_fold.mean(), train_y_fold.std()
                    train_y_fold = (train_y_fold - mu_y) / std_y
                # Creating the train Dataset / DataLoader
                train_dataset_fold = MyDataset(train_X_fold, train_y_fold)
                train_loader_fold = DataLoader(train_dataset_fold, batch_size, shuffle = True)
                
                # Separating the validation data for this fold
                val_X_fold = train_data_X[val_idx[fold_idx], :]
                if single_glycan:
                    val_y_fold = train_data_y[val_idx[fold_idx]]
                else:
                    val_y_fold = train_data_y[val_idx[fold_idx], :]
                # Normalizing the validation set with training set mu and std
                if normalize_X:
                    val_X_fold = (val_X_fold - mu) / std
                if normalize_y:
                    val_y_fold = (val_y_fold - mu_y) / std_y
                # Creating the val Dataset / DataLoader
                val_dataset_fold = MyDataset(val_X_fold, val_y_fold)
                val_loader_fold = DataLoader(val_dataset_fold, batch_size, shuffle = True)

                # Declaring the model and optimizer
                model = SequenceMLP(cur_hp[0], activ_fun).cuda()
                optimizer = torch.optim.Adam(model.parameters(), lr = cur_hp[1], weight_decay = weight_decay)
                scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor = 0.5, patience = 20, verbose = False, min_lr = 1e-5)
                # Train and validate
                for epoch in range(200):
                    train_loss = loop_model(model, optimizer, train_loader_fold, epoch)
                    val_loss = loop_model(model, optimizer, val_loader_fold, epoch, evaluation = True)
                    scheduler.step(val_loss)
                # Recording the validation loss for this fold
                temp_val_loss += val_loss / 4 # Divided by 4 because there are 4 folds

            # Saving the average validation loss after CV
            final_val_loss.at[cur_hp[1], str(cur_hp[0])] = temp_val_loss
            if not nested_validation: # Nested validation requires a bunch of inputs to the same file - I haven't implemented a good way to save the intermetiate data
                final_val_loss.to_csv(osjoin(f'{working_dir}', f'{val_loss_file}'))
    return final_val_loss

In [ ]:
final_val_loss_list = np.empty_like(activ_fun_list, dtype = object) # This will hold multiple DataFrames, one for each activation fun type
for idx, activ_fun in enumerate(activ_fun_list):
    if nested_validation and exp_name == 'NN_modelNSD': # TODO: can likely remove this if-statement from the for loop
        train_idx = np.zeros((5, 4), dtype = object) # 5 nested validation sets, 4 cross validation sets per nested set
        val_idx = np.zeros_like(train_idx)
        # To assist in the setup by marking which set will be the test set
        test_set_list = [set(elem) for elem in [range(0, 4), range(4, 7), range(7, 11), range(11, 15), range(15, 19)]]
        for nested_idx in range(len(test_set_list)):
            list_for_cv = test_set_list[:nested_idx] + test_set_list[nested_idx+1:] # Excluding one set for testing
            train_idx[nested_idx, :] = [list(set(range(19)) - test_set_list[nested_idx] - list_for_cv[fold_idx]) for fold_idx in range(4)]
            val_idx[nested_idx, :] = [list(elem) for elem in list_for_cv]
    elif nested_validation: # Asn_ and Fc_DAO/Fc_EPO with nested validation
        train_idx = np.zeros((5, 4), dtype = object) # 5 nested validation sets, 4 cross validation sets per nested set
        val_idx = np.zeros_like(train_idx)
        # First set: equal to the cross-validation idxs
        train_idx[0, :] = [list( set(range(12)) - set(range(3*fold_idx, 3*(fold_idx+1))) ) for fold_idx in range(4)]
        val_idx[0, :] = [range(3*fold_idx, 3*(fold_idx+1)) for fold_idx in range(4)]
        # Other sets: need to include the last row_idxs in the training / validation (to change the test set)
        for nested_idx in range(1, train_idx.shape[0]):
            temp = list( set(range(15)) - set(range(3*(nested_idx-1), 3*nested_idx)) ) # Excluding the current test set
            train_idx[nested_idx, :] = [temp[:3*fold_idx] + temp[3*(fold_idx+1):] for fold_idx in range(4)]
            val_idx[nested_idx, :] = [temp[3*fold_idx:3*(fold_idx+1)] for fold_idx in range(4)]
    elif exp_name == 'NN_modelNSD':
        # No convenient way to setup the idx
        train_idx = [list(range(4, 15)), list(range(0, 4)) + list(range(7, 15)), list(range(0, 7)) + list(range(11, 15)), list(range(0, 12))]
        val_idx = [list( set(range(15)) - set(train_idx[fold_idx]) ) for fold_idx in range(4)]
    else: # Asn_ and Fc_DAO/Fc_EPO
        train_idx = [list( set(range(12)) - set(range(3*fold_idx, 3*(fold_idx+1))) ) for fold_idx in range(4)]
        val_idx = [range(3*fold_idx, 3*(fold_idx+1)) for fold_idx in range(4)]

    working_dir = generate_working_dir(activ_fun, normalize_X, normalize_y, nested_validation) # Results folder
    # Running the CV (no nested validation) / groups of CVs (nested validation)
    if nested_validation:
        print(f'Nested validation: beginning CV loop 1/{train_idx.shape[0]}')
        final_val_loss_list[idx] = CV_model(activ_fun, working_dir, val_loss_file, train_idx[0, :], val_idx[0, :])
        if not isfile(osjoin(f'{working_dir}', f'{val_loss_file}')): # Avoid replacing (and thus incorrectly increasing the losses every run)
            for cv_idx in range(1, train_idx.shape[0]):
                print(f'Nested validation: beginning CV loop {cv_idx+1}/{train_idx.shape[0]}')
                final_val_loss_list[idx] += CV_model(activ_fun, working_dir, val_loss_file, train_idx[cv_idx, :], val_idx[cv_idx, :])
            # Saving the results after all nested validation runs (for non-nested, save occurs within the CV_model function)
            final_val_loss_list[idx].to_csv(osjoin(f'{working_dir}', f'{val_loss_file}'))
    else:
        final_val_loss_list[idx] = CV_model(activ_fun, working_dir, val_loss_file, train_idx, val_idx)

## Final Evaluation - Testing the best model

In [ ]:
def run_final_evaluation(model, working_dir, activ_fun):
    model.eval()

    # Train loss
    if single_glycan:
        train_pred = torch.empty(len(train_loader.dataset))
    else:
        train_pred = torch.empty((len(train_loader.dataset), myshape_y))
    train_y = torch.empty_like(train_pred)
    for idx, data in enumerate(train_loader):
        X, y = data
        X = X.cuda()
        pred = model(X).cpu().detach().squeeze()
        if single_glycan:
            train_pred[idx*batch_size:(idx*batch_size)+len(pred)] = pred
            train_y[idx*batch_size:(idx*batch_size)+len(y)] = y
        else:
            train_pred[idx*batch_size:(idx*batch_size)+len(pred), :] = pred
            train_y[idx*batch_size:(idx*batch_size)+len(y), :] = y
    #train_loss = torch.nn.functional.mse_loss(train_pred, train_y)
    #print(f'The train loss was {train_loss.item():.2e}')
    # Train predictions plot
    if single_glycan:
        print(f'Predicted training mean {train_pred.mean():.3f} ±{train_pred.std():.3f}, real training mean {train_y.mean():.3f}')
        if normalize_y:
            print(f'\tDenormalized: predicted {train_pred.mean()*std_y + mu_y:.3f}, real {train_y.mean()*std_y + mu_y:.3f}')
    else:
        fig, ax = plt.subplots(figsize = (8,8))
        ax.set_title('Train Predictions - mean')
        plt.scatter(train_y.mean(axis = 0), train_pred.mean(axis = 0))
        ax.set_xlabel('Real Values')
        ax.set_ylabel('Predicted Values')
    fig, ax = plt.subplots(figsize = (8,8))
    ax.set_title(f'{activ_fun}: Train Predictions')
    plt.scatter(train_y, train_pred)
    ax.set_xlabel('Real Values')
    ax.set_ylabel('Predicted Values')
    axis_min_lim = torch.minimum(train_pred.min(), train_y.min())
    axis_min_lim = np.minimum(axis_min_lim, 0) # If all values are positive, start from 0
    axis_max_lim = torch.maximum(train_pred.max(), train_y.max())
    ax.set_xlim(axis_min_lim - 0.01, axis_max_lim + 0.01)
    ax.set_ylim(axis_min_lim - 0.01, axis_max_lim + 0.01)
    # Perfect prediction line (just a 45° line)
    plt.plot([axis_min_lim, axis_max_lim], [axis_min_lim, axis_max_lim], color = 'k', linestyle = '--', linewidth = 1)
    # 10% relative error lines
    plt.plot([axis_min_lim, axis_max_lim], [1.1*axis_min_lim, 1.1*axis_max_lim], color = 'r', linestyle = '--', linewidth = 1)
    plt.plot([axis_min_lim, axis_max_lim], [0.9*axis_min_lim, 0.9*axis_max_lim], color = 'r', linestyle = '--', linewidth = 1)
    # Test loss
    if single_glycan:
        test_pred = torch.empty(len(test_loader.dataset))
    else:
        test_pred = torch.empty((len(test_loader.dataset), myshape_y))
    test_y = torch.empty_like(test_pred)
    for idx, data in enumerate(test_loader):
        X, y = data
        X = X.cuda()
        pred = model(X).cpu().detach().squeeze()
        if single_glycan:
            test_pred[idx*batch_size:(idx*batch_size)+len(pred)] = pred
            test_y[idx*batch_size:(idx*batch_size)+len(y)] = y
        else:
            test_pred[idx*batch_size:(idx*batch_size)+len(pred), :] = pred
            test_y[idx*batch_size:(idx*batch_size)+len(y), :] = y
    #test_loss = torch.nn.functional.mse_loss(test_pred, test_y)
    #print(f'The test loss was {test_loss:.2e}')
    # Test predictions plot
    if single_glycan:
        print(f'Predicted test mean {test_pred.mean():.3f} ±{test_pred.std():.3f}; real test mean {test_y.mean():.3f}')
        if normalize_y:
            print(f'\tDenormalized: predicted {test_pred.mean()*std_y + mu_y:.3f}, real {test_y.mean()*std_y + mu_y:.3f}')
    else:
        fig, ax = plt.subplots(figsize = (8,8))
        ax.set_title('Test Predictions - mean')
        plt.scatter(test_y.mean(axis = 0), test_pred.mean(axis = 0))
        ax.set_xlabel('Real Values')
        ax.set_ylabel('Predicted Values')
        # Changing the axis limits for easier visualization
        axis_lim = torch.maximum(test_pred.mean(axis=0).max(), test_y.mean(axis=0).max())
        ax.set_xlim(-0.01, axis_lim + 0.01)
        ax.set_ylim(-0.01, axis_lim + 0.01)
        # Saving the test mean predictions for comparison
        pd.DataFrame(test_pred.mean(axis = 0)).T.to_csv(osjoin(f'{working_dir}', f'temp_{exp_name}.csv'), header = False, index = False)

In [ ]:
# Removing the test set from the training data for the final evaluation
if nested_validation and exp_name == 'NN_modelNSD' and train_data_X.shape[0] == 19:
    train_data_X = train_data_X[:-4]
    train_data_y = train_data_y[:-4]
elif nested_validation and exp_name != 'NN_modelNSD' and train_data_X.shape[0] == 15:
    train_data_X = train_data_X[:-3]
    train_data_y = train_data_y[:-3]

# Normalizing the training set
if normalize_X:
    mu, std = train_data_X.mean(), train_data_X.std()
    train_data_X = (train_data_X - mu) / std
if normalize_y:
    mu_y, std_y = train_data_y.mean(), train_data_y.std()
    train_data_y = (train_data_y - mu_y) / std_y
# Creating the full training Dataset / DataLoader
train_dataset = MyDataset(train_data_X, train_data_y)
train_loader = DataLoader(train_dataset, batch_size, shuffle = True)
# Normalizing the test set with training set mu and std
if normalize_X:
    test_data_X = (test_data_X - mu) / std
if normalize_y:
    test_data_y = (test_data_y - mu_y) / std_y
# Creating the test Dataset / DataLoader
test_dataset = MyDataset(test_data_X, test_data_y)
test_loader = DataLoader(test_dataset, batch_size, shuffle = True)

print(f'Current glycan: {exp_name}-{single_glycan}')
for final_val_loss, activ_fun in zip(final_val_loss_list, activ_fun_list):
    working_dir = generate_working_dir(activ_fun, normalize_X, normalize_y, nested_validation) # Results folder
    # Finding the best hyperparameters
    best_idx = np.argmin(final_val_loss.values.T) # .T to ensure proper ordering, as argmin flattens the array
    # Re-declaring the model
    model = SequenceMLP(hyperparam_list[best_idx][0], activ_fun).cuda()

    # Checking if we already retrained this model
    try:
        mydict = torch.load(osjoin(f'{working_dir}', f'{best_model_file}'))
        model.load_state_dict(mydict)
    except FileNotFoundError: # Retraining the model with the full training set
        optimizer = torch.optim.Adam(model.parameters(), lr = hyperparam_list[best_idx][1], weight_decay = weight_decay)
        # Retrain
        for epoch in range(200):
            train_loss = loop_model(model, optimizer, train_loader, epoch)
        # Save the retrained model
        torch.save(model.state_dict(), osjoin(f'{working_dir}', f'{best_model_file}'))
    
    # CV Data
    print(f'Final results for {activ_fun} and weight decay {weight_decay:.1e}')
    print(f'Best hyperparameters: {hyperparam_list[best_idx]}')
    print(f'CV Loss: {final_val_loss.at[ hyperparam_list[best_idx][1], str(hyperparam_list[best_idx][0]) ]:.5e}')
    run_final_evaluation(model, working_dir, activ_fun)
    print()